<p> This notebook goes through the process of using a supervised learning algorithm to assign classification labels to Census tracts in the Atlanta region. Next, the code spatially indexes and then aggregates home sales from 2018 and 2023 to the tract level, joining these data with other metrics at the same geography.</p>


### 1 - Dependencies & global variables


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import joblib
import numpy as np

# define list of metro ATL counties
atl_FIPS = {
    'Cherokee': '13057',
    'Clayton': '13063',
    'Cobb': '13067',
    'DeKalb': '13089',
    'Douglas': '13097',
    'Fayette': '13113',
    'Forsyth': '13117',
    'Fulton': '13121',
    'Gwinnett': '13135',
    'Henry': '13151',
    'Rockdale': '13247'
}

# create inverse dictionary
inverse_FIPS_dict = {value: key for key, value in atl_FIPS.items()}

# pre-define the urban counties on which to run the supervised learning model
urban_counties = ['13063', '13121', '13089', '13135', '13067']

# pre-define rural counties, which are determined by Census designation (i.e., no supervised learning model)
rural_counties = ['13057', '13097', '13113', '13117', '13151', '13247']

# Load 2020 tract geometries into memory via GeoDataFrames
url_tracts_2020 = "https://www2.census.gov/geo/tiger/TIGER2024/TRACT/tl_2024_13_tract.zip"
GA_2020_Tracts = gpd.read_file(url_tracts_2020).to_crs(epsg=26916)
GA_2020_Tracts['FIPS'] = GA_2020_Tracts['STATEFP'] + GA_2020_Tracts['COUNTYFP']
GA_2020_Tracts = GA_2020_Tracts[[
    'GEOID',
    'FIPS',
    'geometry'
]]
GA_2020_Tracts = GA_2020_Tracts[GA_2020_Tracts['FIPS'].isin(atl_FIPS.values())]

# Load 2010 tract geometries into memory via GeoDataFrames
url_tracts_2010 = "https://www2.census.gov/geo/tiger/TIGER2018/TRACT/tl_2018_13_tract.zip"
GA_2010_Tracts = gpd.read_file(url_tracts_2010).to_crs(epsg=26916)
GA_2010_Tracts['FIPS'] = GA_2010_Tracts['STATEFP'] + GA_2010_Tracts['COUNTYFP']
GA_2010_Tracts = GA_2010_Tracts[[
    'FIPS',
    'GEOID',
    'geometry'
]]
GA_2010_Tracts = GA_2010_Tracts[GA_2010_Tracts['FIPS'].isin(atl_FIPS.values())]

# Read in land use classification labels (urban-suburban-rural) used previously for ML model
landUse_labels = pd.read_csv(
    'Data/tracts_by_submarket.csv',
    dtype={
        'GEOID': 'str'
    })
# should be in the format of TRACT_ID [string] | Submarket [string]

# Read in tract-level land use percentages for 2018
landUse_proportions_2018 = pd.read_csv(
    'Data/LandUse_ByTract_2018.csv', dtype={'GEOID': 'str'})
# should be in the structure of TRACT_ID [string] | Ag_LandUsePercentage [float] \ Comm_LandUsePercentage [float]...

# Read in tract-level land use percentages for 2023
landUse_proportions_2023 = pd.read_csv(
    'Data/LandUse_ByTract_2023.csv', dtype={'GEOID': 'str'})
# same structure as above

# Define the keywords to include, remove from the LandUse_ByTract files above
# Certain land uses may adversely affect the model and may need to be filtered out
land_uses = [
    'GEOID',
    'Commercial Office',
    'Commercial Office - 2+ Floors',
    'Commercial Office - Mixed Use',
    'Commercial Retail',
    # 'Commercial Retail - Condomium',
    'Commercial Retail - Department Store',
    'Commercial Retail - Food & Drink',
    # 'Commercial Retail - Gas and Service Station',
    # 'Commercial Retail - Hotel, Motel, Resort',
    # 'Commercial Retail - Liquor, Convenience Store',
    'Commercial Retail - Mall',
    # 'Commercial Retail - Park Lot, Garage',
    'Commercial Retail - Shopping Plaza, Mini-Mall',
    'Commercial Retail - Truck Stop',
    # 'Commercial Retail - Vet, Animal Hospital',
    'Multi-Family Residential',
    'Multi-Family Residential - 2 to 4 units',
    'Multi-Family Residential - 5+ units',
    'Multi-Family Residential - High-Rise',
    # 'Multi-Family Residential - Mobile Home Park',
    'Residential  ',
    'Residential - Condo',
    # 'Residential - Mobile Home Park',
    # 'Residential - Parking Garage',
    'Residential - Townhouse'
]

# Select filtered columns to make dataframe with only residential & commercial
landUse_proportions_2018 = landUse_proportions_2018[land_uses]
landUse_proportions_2023 = landUse_proportions_2023[land_uses]

# Read in the rural tracts as designated by Census
percent_rural = pd.read_csv('Data/percent_rural.csv', dtype={'GEOID': 'str'})

### 2 - Calculate distance to urban core for both training & testing data


In [ ]:
# Function to calculate distance from a reference point
def calculate_distance_to_core(tracts, core_point_projected):
    tracts['centroid'] = tracts.centroid
    tracts['distance_from_core'] = tracts['centroid'].distance(
        core_point_projected)
    return tracts


# Function to merge datasets and clean up columns
def merge_and_clean_data(land_use_df, tracts_df, drop_columns, key='GEOID'):
    merged_df = pd.merge(land_use_df, tracts_df, how='left', on=key)
    merged_df = merged_df.drop(columns=drop_columns)
    return merged_df


# Set lat / long values to some point in CBD (in this case, downtown Atlanta)
core_lat, core_lon = 33.76, -84.39
core_point = Point(core_lon, core_lat)
core_point_projected = gpd.GeoSeries(
    [core_point], crs="EPSG:4326").to_crs(epsg=26916).iloc[0]

# Drop unneeded columns
drop_columns = ['FIPS', 'geometry', 'centroid']

# Process 2018 dataset
GA_2010_Tracts = calculate_distance_to_core(
    GA_2010_Tracts, core_point_projected)
distance_df_2018 = merge_and_clean_data(
    landUse_proportions_2018, GA_2010_Tracts, drop_columns)

# Process 2023 dataset
GA_2020_Tracts = calculate_distance_to_core(
    GA_2020_Tracts, core_point_projected)
distance_df_2023 = merge_and_clean_data(
    landUse_proportions_2023, GA_2020_Tracts, drop_columns)

# Print results
print(f'rows in 2018 dataset: {distance_df_2018.shape[0]:,}')
print(f'rows in 2023 dataset: {distance_df_2023.shape[0]:,}')
distance_df_2023.head(3)

rows in 2018 dataset: 782
rows in 2023 dataset: 1,247


,GEOID,Commercial Office,Commercial Office - 2+ Floors,Commercial Office - Mixed Use,Commercial Retail,Commercial Retail - Department Store,Commercial Retail - Food & Drink,Commercial Retail - Mall,"Commercial Retail - Shopping Plaza, Mini-Mall",Commercial Retail - Truck Stop,Multi-Family Residential,Multi-Family Residential - 2 to 4 units,Multi-Family Residential - 5+ units,Multi-Family Residential - High-Rise,Residential,Residential - Condo,Residential - Townhouse,distance_from_core
0,13057090101,0.004981,0.0,0.0,0.012453,0.0,0.003113,0.000623,0.0,0.0,0.0,0.001245,0.0,0.0,0.893524,0.0,0.000000,65619.565436
1,13057090102,0.001856,0.0,0.0,0.001856,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.941067,0.0,0.000000,64648.339127
2,13057090103,0.005302,0.0,0.0,0.015907,0.0,0.001060,0.000000,0.0,0.0,0.0,0.001060,0.0,0.0,0.843054,0.0,0.025451,61638.095114


### 3 - X, Y splits -> train -> run model for urban counties


In [ ]:
urban_dfs = []

# will run the model on just the urban counties as defined above
for county in urban_counties:

    # filter 2018 data
    groupby = distance_df_2018[distance_df_2018['GEOID'].str.startswith(
        county)]

    merged_df_labels = pd.merge(
        groupby,
        landUse_labels,
        how='inner',
        on='GEOID'
    )

    # clean up the dataframe
    merged_df_labels = merged_df_labels.drop(columns='Submarket')

    # move the classification column from the last to the second position
    new_order = merged_df_labels.columns.tolist()
    new_order.insert(1, new_order.pop())
    merged_df_labels = merged_df_labels.reindex(columns=new_order)

    # rename label column
    merged_df_labels = merged_df_labels.rename(columns={
        'General Landuse Type': 'label'
    })

    # remove rural designation from the test dataset
    merged_df_labels = merged_df_labels[merged_df_labels['label'] != 'Rural']

    # Separate the features and labels
    X = merged_df_labels.drop(columns=['GEOID', 'label'])
    y = merged_df_labels['label']

    # define a scaler, use it to fit & transform
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # save scaler as pickle file
    joblib.dump(
        scaler, f'Pickle/scaler_{inverse_FIPS_dict[county].lower()}.pkl')

    # split the dataset
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled,
        y,
        test_size=0.2,
        random_state=42
    )

    # Train the model with the best parameters
    model = DecisionTreeClassifier(
        criterion='entropy',
        splitter='random',
        max_features=16,
        random_state=1
    )
    model.fit(X_train, y_train)

    # Save the trained model to .pkl
    joblib.dump(
        model,
        f'Pickle/decision_tree_model_{inverse_FIPS_dict[county].lower()}.pkl'
    )

    # Predict on the test set
    y_pred = model.predict(X_test)

    # prediction time for 2023 data ----------------------------------------------------

    # load saved pickle files
    scaler = joblib.load(
        f'Pickle/scaler_{inverse_FIPS_dict[county].lower()}.pkl')
    model = joblib.load(
        f'Pickle/decision_tree_model_{inverse_FIPS_dict[county].lower()}.pkl')

    # filter 2023 data
    filtered_2023_data = distance_df_2023[distance_df_2023['GEOID'].str.startswith(
        county)]

    # drop the tract ID column so we are only left with land use proportions
    X_new = filtered_2023_data.drop(columns='GEOID')

    # scale to the new 2023 data
    X_new_scaled = scaler.transform(X_new)

    # Predict using the loaded model
    y_new_pred = model.predict(X_new_scaled)

    # add predictions
    model_df_23 = filtered_2023_data.copy()
    model_df_23['label'] = y_new_pred

    # slim down the dataframe
    model_df_23 = model_df_23[[
        'GEOID',
        'label'
    ]]

    # Add this dataframe to the 'urban_dfs' dataframe
    urban_dfs.append(model_df_23)

combined_df = pd.concat(urban_dfs, ignore_index=True)

export complete!


### 4 - Label all tracts with proper designations


In [ ]:
# Add in the rest of metro tracts to 'combined_df'
new_geoids = GA_2020_Tracts[~GA_2020_Tracts['GEOID'].isin(
    combined_df['GEOID'])]
new_rows = new_geoids.copy()
new_rows['label'] = 'TBD'
new_rows = new_rows[[
    'GEOID',
    'label'
]]

# Append the new rows to combined_df
decisiontree_combined = pd.concat([combined_df, new_rows], ignore_index=True)

# join the percent rural to this dataframe and override the label if rural > 50
decisiontree_combined = decisiontree_combined.merge(
    percent_rural,
    how='left',
    on='GEOID'
).drop(columns=['ALAND', 'Total Urban Area'])

# Step 1: Update 'label' to "Rural" where 'Percent Rural' is >= 50
decisiontree_combined.loc[decisiontree_combined['Percent Rural']
                          >= 50, 'label'] = 'Rural'

# Step 2: Change remaining "TBD" values in 'label' to "Suburban"
decisiontree_combined.loc[decisiontree_combined['label']
                          == 'TBD', 'label'] = 'Suburban'

# Add geometry👇; stays the same for all techniques
gdf_DecisionTree_urban = gpd.GeoDataFrame(pd.merge(
    decisiontree_combined,
    GA_2020_Tracts,
    how='left',
    left_on='GEOID',
    right_on='GEOID'
))
gdf_DecisionTree_urban


# smoothing technique for suburban tracts --------------------------------------------
# Function to calculate the shared boundary length
def shared_boundary_length(suburban_geom, urban_sindex, urban_tracts):
    possible_matches_index = list(
        urban_sindex.intersection(suburban_geom.bounds))
    possible_matches = urban_tracts.iloc[possible_matches_index]
    possible_matches = possible_matches[possible_matches.intersects(
        suburban_geom)]
    shared_length = sum(possible_matches.intersection(suburban_geom).length)
    return shared_length


# Smoothing function
def smooth_labels(gdf):
    # Reproject to a suitable projected CRS (NAD83 / Georgia West)
    gdf = gdf.to_crs(epsg=2240)

    suburban_tracts = gdf[gdf['label'] == 'Suburban'].copy()
    urban_tracts = gdf[gdf['label'] == 'Urban'].copy()

    # Create spatial index for urban tracts for faster querying
    urban_sindex = urban_tracts.sindex

    # Calculate the shared boundary length for each suburban tract
    suburban_tracts['shared_length'] = suburban_tracts.geometry.apply(
        lambda geom: shared_boundary_length(geom, urban_sindex, urban_tracts))

    # Calculate the total boundary length of each suburban tract
    suburban_tracts['total_length'] = suburban_tracts.geometry.length

    # Calculate the percentage of boundary touching urban tracts
    suburban_tracts['percentage_touching_urban'] = suburban_tracts['shared_length'] / \
        suburban_tracts['total_length']

    # Update the label for tracts where more than 50% of the boundary touches urban tracts
    suburban_tracts.loc[suburban_tracts['percentage_touching_urban']
                        > 0.5, 'label'] = 'Urban'

    # Merge back the updated suburban tracts with the original GeoDataFrame
    gdf.update(suburban_tracts)

    return gdf


# function that will run the smoothing process a specified number of times
def run_smoothing(gdf, iterations):
    for i in range(iterations):
        gdf = smooth_labels(gdf)
    return gdf


# Number of times to run the smoothing function
iterations = 2

# Run the smoothing function
gdf_DecisionTree_urban = run_smoothing(gdf_DecisionTree_urban, iterations)

# end of tract smoothing technique--------------------------------------------------

# Slim down & export
gdf_DecisionTree_urban_export = gdf_DecisionTree_urban[[
    'GEOID',
    'label'
]]
gdf_DecisionTree_urban_export.to_csv('Data/tracts_with_labels.csv')
print('export complete!')

### 5 - Process home sales data


In [ ]:
# read in unaggregated sales
sales = pd.read_csv(f'../4_DataExport/residentialSales_Updated.csv')

# Convert DataFrame to GeoDataFrame so we can spatially join
sales = gpd.GeoDataFrame(
    sales,
    geometry=gpd.points_from_xy(
        sales['long'], sales['lat']),
    crs="EPSG:4269"
).to_crs(epsg=26967)

# spatially join sales to geography of choice
sales = gpd.sjoin(
    sales,
    GA_2020_Tracts,
    how='left',
    predicate='within'
).drop(columns='index_right')

# if a sale didn't join to a Census tract, drop it
sales = sales[sales['GEOID'].notna()]

# using the GEOID column, create a 'county' column
sales['county'] = sales['GEOID'].str[:5].map(inverse_FIPS_dict)

# pare down & reorder sales
sales = sales[[
    'ATTOM_ID',  # keeping this column so we can remove duplicate sales of the same property
    'county',
    'GEOID',
    'sale_date',
    'year',
    'month',
    'sale_amt',
    'price_sf',
    'home_size',
    'yr_blt',
    'res_type',
    'lat',
    'long'
]]

# Group by both ATTOM_ID and year to count occurrences for each group
grouped_counts_df = sales.groupby(
    ['ATTOM_ID', 'year']).size().reset_index(name='sale_count')

# Merge the grouped counts back into the original 'sales' DataFrame
sales = sales.merge(grouped_counts_df, on=['ATTOM_ID', 'year'], how='left')


# define function to compute the cleaned price / SF, which will calculate an "aggregated" price / sf if a property has sold more than once in a given year
def clean_price_sf(row):
    attom_id = row['ATTOM_ID']
    year = row['year']
    count = row['sale_count']

    if count == 1:
        return row['price_sf']
    else:
        subset = sales[(sales['ATTOM_ID'] == attom_id)
                       & (sales['year'] == year)]
        if count <= 3:
            return subset['price_sf'].max()
        else:
            return subset['price_sf'].median()


# Apply the de-dupe function defined above to create the 'price_sf_cleaned' column
sales = sales.copy()
sales['price_sf_cleaned'] = sales.apply(clean_price_sf, axis=1)

# Now that duplicate properties share a common price / SF, we can drop any dupes (while keeping the first instance)
sales = sales.sort_values(by=['ATTOM_ID', 'year'])
sales = sales.drop_duplicates(subset=['ATTOM_ID', 'year'], keep='first')


# 2-step to drop the original 'price_sf' column and then rename the cleaned 'price_sf' back to its original name
sales = sales.drop(columns='price_sf').rename(columns={
    'price_sf_cleaned': 'price_sf'
})

# map residential type descriptions
res_type_dict = {
    385.0: 'single-family residence',
    383.0: 'single-family residence',
    380.0: 'single-family residence',
    363.0: 'single-family residence',
    366.0: 'condominium',
    386.0: 'townhouse',
    382.0: 'townhouse',
    369.0: 'small multi-family',
    388.0: 'small multi-family',
    378.0: 'small multi-family',
    373.0: 'mobile home',
    371.0: 'modular',
    364.0: 'other'
}

# create new column with short descriptions
sales['res_type_desc'] = sales['res_type'].map(res_type_dict).fillna('unknown')

# drop the numeric 3-digit residential type column
sales = sales.drop(columns='res_type')

# For comparison's sake, this will create a tract-level rollup without any filters applied
sales_23 = sales[sales['year'] == 2023]
sales_18 = sales[sales['year'] == 2018]
sales_years = list(sales['year'].unique())


# define the modified z-score outliers function
def filter_by_modified_zscore(data, threshold=3.5):
    median = np.median(data)
    mad = np.median(np.abs(data - median))
    if mad == 0:
        return data  # No outliers if mad is 0
    modified_z_scores = 0.6745 * (data - median) / mad
    return data[np.abs(modified_z_scores) < threshold]


# define the percentile outliers filter function
def filter_by_percentiles(data, lower_percentile, upper_percentile):
    lower_bound = np.percentile(data, lower_percentile)
    upper_bound = np.percentile(data, upper_percentile)
    return data[(data >= lower_bound) & (data <= upper_bound)]


# define function to filter data by each method
def filter_group(group, method, **kwargs):
    prices = group['price_sf']
    if method == 'modified_zscore':
        filtered_prices = filter_by_modified_zscore(prices, **kwargs)
    elif method == 'percentiles':
        filtered_prices = filter_by_percentiles(prices, **kwargs)
    return group[group['price_sf'].isin(filtered_prices)]


# apply the filter function above to the dataset for each year, for each Census tract
sales_list = []
for year in sales_years:
    df_year = sales[sales['year'] == year]

    filtered_sales_zscore = df_year.groupby('GEOID', group_keys=False).apply(
        filter_group,
        method='modified_zscore',
        threshold=3.5,
        include_groups=True
    ).reset_index(drop=True)

    sales_list.append(filtered_sales_zscore)

# create cleaned dataframe, inclusive of all years
combined_cleaned_full_df = pd.concat(sales_list, ignore_index=False)

# create cleaned dataframes of just 2018, 2023
filtered_sales_zscore_23 = sales_23.groupby('GEOID').apply(
    filter_group,
    method='modified_zscore',
    threshold=3.5,
    include_groups=False
).reset_index(drop=False).drop(columns='level_1')
filtered_sales_zscore_18 = sales_18.groupby('GEOID').apply(
    filter_group,
    method='modified_zscore',
    threshold=3.5,
    include_groups=False
).reset_index(drop=False).drop(columns='level_1')

# create tract-level summary dataframes
sales_23_grouped = filtered_sales_zscore_23.groupby(
    'GEOID')['price_sf'].median().reset_index()
sales_18_grouped = filtered_sales_zscore_18.groupby(
    'GEOID')['price_sf'].median().reset_index()

# merge 2018 and 2023 dataframes
sales_finalMerge = pd.merge(
    sales_18_grouped,
    sales_23_grouped,
    how='outer',
    left_on='GEOID',
    right_on='GEOID',
    suffixes=('_18', '_23')
)

# calculate percent change columns (numeric & percentage)
sales_finalMerge['5yrChange_num'] = sales_finalMerge['price_sf_23'] - \
    sales_finalMerge['price_sf_18']
sales_finalMerge['5yrChange_per'] = (sales_finalMerge['price_sf_23'] -
                                     sales_finalMerge['price_sf_18']) / sales_finalMerge['price_sf_18'] * 100


sales_finalMerge.to_csv('4_DataExport/submarket_sales_4.csv', index=False)
print('export complete!')
print(f'rows in 2018 & 2023 dataset: {sales_finalMerge.shape[0]:,}')
sales_finalMerge.sample(5)

### 6 - Load & merge data sources


In [ ]:
# read in sales data
sales_df = pd.read_csv(
    '../4_DataExport/submarket_sales_4.csv',
    dtype={
        'GEOID': 'str'
    })

# read in the urban / suburban / rural labels
labels_df = pd.read_csv(
    'Supervised/tracts_with_labels.csv',
    dtype={
        'GEOID': 'str'
    })

# CSV containing job-to-pop ratio
jobPop_ratio = pd.read_csv(
    'submarket_analysis.csv',
    dtype={
        'GEOID': 'str'
    })

jobPop_ratio = jobPop_ratio[[
    'GEOID',
    'Job_to_pop_ratio'
]]

# get town centers
url = 'https://services1.arcgis.com/Ug5xGQbHsD8zuZzM/arcgis/rest/services/2019_UGPM_WFL1/FeatureServer/4/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&returnCentroid=false&returnEnvelope=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnTrueCurves=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token='

town_centers = gpd.read_file(url)
town_centers = town_centers.to_crs(GA_2020_Tracts.crs)

# Perform a spatial join to find intersections
intersected = gpd.sjoin(GA_2020_Tracts, town_centers,
                        how='left', predicate='intersects')
intersected = intersected.drop_duplicates(subset='GEOID')

# Create the 'town_center' column based on whether an intersection was found
GA_2020_Tracts['in_town_center'] = intersected['index_right'].notnull()

GA_2020_Tracts = GA_2020_Tracts[[
    'GEOID',
    'in_town_center',
]]

# merge the 2 above labels
final_df1 = pd.merge(
    sales_df,
    labels_df,
    how='right',
    on='GEOID'
).drop(columns=['5yrChange_num', '5yrChange_per'])

# drop any tracts where there was no 2018 or 2023 sales data
final_df1 = final_df1[final_df1['price_sf_23'].notna()]
final_df1 = final_df1[final_df1['price_sf_18'].notna()]

# merge with the job/pop ratio dataframe
final_df2 = final_df1.merge(
    jobPop_ratio,
    how='left',
    on='GEOID'
)

# merge with town center dataframe
final_df3 = final_df2.merge(
    GA_2020_Tracts,
    on='GEOID'
)
final_df3 = final_df3[[
    'GEOID',
    'label',
    'in_town_center',
    'Job_to_pop_ratio',
    'price_sf_18',
    'price_sf_23'
]]

summ_df = final_df3.groupby('label')['price_sf_23'].median().reset_index()

final_df4 = final_df3.merge(
    summ_df,
    on='label'
).rename(columns={
    'price_sf_23_x': 'price_sf_23',
    'price_sf_23_y': 'label_median'
})

final_df4['price_desc'] = final_df4.apply(
    lambda row: 'high priced' if row['price_sf_23'] >= row['label_median'] else 'low priced',
    axis=1
)

final_df4.sample(5)

### 7 - Finally, apply submarket labels


In [ ]:
# create copy
final_df5 = final_df4.copy()


# Function to calculate percent change when it becomes needed
def calculate_percent_change(row):
    return ((row['price_sf_23'] - row['price_sf_18']) / row['price_sf_18']) * 100


# Calculate percent change for 'Urban' tracts
final_df5['percent_change'] = final_df5.apply(lambda row: calculate_percent_change(
    row) if row['label'] == 'Urban' else 'N/A', axis=1)

# Calculate the median percent change for 'Urban' records
median_percent_change = final_df5[final_df5['label']
                                  == 'Urban']['percent_change'].median()


# Define the function to apply the logic
def assign_submarket(row):
    if row['label'] == 'Rural':
        if row['price_desc'] == 'high priced':
            return 10
        elif row['price_desc'] == 'low priced':
            return 9
    elif row['label'] == 'Urban':
        if row['price_desc'] == 'high priced':
            if row['Job_to_pop_ratio'] >= 1:
                return 2
            else:
                return 1
        elif row['price_desc'] == 'low priced':
            if row['percent_change'] > median_percent_change:
                return 3
            else:
                return 4
    elif row['label'] == 'Suburban':
        if (row['Job_to_pop_ratio'] >= 1 or row['in_town_center']) and row['price_desc'] == 'high priced':
            return 5
        elif (row['Job_to_pop_ratio'] >= 1 or row['in_town_center']) and row['price_desc'] == 'low priced':
            return 8
        elif not row['in_town_center'] and row['price_desc'] == 'high priced':
            return 6
        elif not row['in_town_center'] and row['price_desc'] == 'low priced':
            return 7
    return None  # In case none of the conditions match


# Apply the function to create the 'submarket' column
final_df5['submarket'] = final_df5.apply(assign_submarket, axis=1)

# Drop the temporary 'percent_change' column
final_df5.drop(columns=['percent_change'], inplace=True)

print(
    f'median change in urban tracts from 18 to 23: {median_percent_change:.1f}%')

# export to CSV
final_df5.to_csv('submarket_by_tract.csv', index=False)